# IUM projekt - temat 2

## Treść zadania
> Są osoby, które wchodzą na naszą stronę i nie mogą się zdecydować, którym produktom przyjrzeć się nieco lepiej. Może dało by się im coś polecić?

### Obliczenia
Poniżej znajdują się obliczenia, na których oparte zostały wartości liczbowe z powyższego raportu. Wszystkie obliczenia dotyczą bazy rekordów wydarzeń sesji.

In [6]:
import numpy as np
import pandas as pd

data_raw_dir = 'data_raw/'
data_dir = 'data/'
sessions_file = 'sessions.jsonl'
sessions_raw_filepath = data_raw_dir + sessions_file
sessions_filepath = data_dir + sessions_file

In [7]:
# Dane wydarzeń sesji
sessions_raw_data = pd.read_json(sessions_raw_filepath, lines=True)
sessions_raw_data

,session_id,timestamp,user_id,product_id,event_type,offered_discount,purchase_id
0,100001,2021-01-26 00:42:30,NaN,1001.0,VIEW_PRODUCT,15,NaN
1,100002,2021-01-05 04:28:35,102.0,1308.0,VIEW_PRODUCT,5,NaN
2,100002,2021-01-05 04:29:28,102.0,1297.0,VIEW_PRODUCT,5,NaN
3,100002,2021-01-05 04:31:43,102.0,1291.0,VIEW_PRODUCT,5,NaN
4,100002,2021-01-05 04:34:16,102.0,1313.0,VIEW_PRODUCT,5,NaN
...,...,...,...,...,...,...,...
87598,110658,2021-03-11 13:07:51,301.0,1048.0,VIEW_PRODUCT,10,NaN
87599,110658,2021-03-11 13:09:45,301.0,1143.0,VIEW_PRODUCT,10,NaN
87600,110658,2021-03-11 13:11:31,301.0,1110.0,VIEW_PRODUCT,10,NaN
87601,110658,2021-03-11 13:13:14,301.0,1141.0,VIEW_PRODUCT,10,NaN


#### Ilość błędnych wierszy w bazie rekordów sesji

In [8]:
row_num_pre_drop = len(sessions_raw_data.index)
print('Ilość wierszy przed usunięciem błędnych danych: ', row_num_pre_drop)

data_cleaned = pd.read_json(sessions_filepath, lines=True)
row_num_after_drop = len(data_cleaned.index)
print('Ilość wierszy po odzyskaniu i usunięciu błędnych danych: ', row_num_after_drop)

fraction_lost = (row_num_pre_drop - row_num_after_drop) / row_num_pre_drop
print('Usuniętych danych: {:.2%}'.format(fraction_lost))

Ilość wierszy przed usunięciem błędnych danych:  87603
Ilość wierszy po odzyskaniu i usunięciu błędnych danych:  83034
Usuniętych danych: 5.22%


#### Średnia ilość przejrzeń produktów dziennie
Wliczane są dane uszkodzone, tzn. również te, w których brakuje części danych.

In [9]:
# Ilość wyświetleń na dzień
view_event_timestamps = sessions_raw_data[sessions_raw_data['event_type']=='VIEW_PRODUCT']['timestamp']
view_events_per_day = (pd.to_datetime(view_event_timestamps)
    .dt.floor('d')
    .value_counts()
    .rename_axis('day')
    .reset_index(name='count')
    .sort_values('day'))
view_events_per_day

,day,count
3,2021-01-01,1484
29,2021-01-02,1155
0,2021-01-03,1663
13,2021-01-04,1316
11,2021-01-05,1337
...,...,...
14,2021-03-11,1314
28,2021-03-12,1179
2,2021-03-13,1614
12,2021-03-14,1325


In [10]:
# Średnia ilość wyświetleń na dzień
mean_view_events_per_day = view_events_per_day['count'][:-1].mean()
round(mean_view_events_per_day)

1123

Stąd wynika, że wzrost wyświetleń o 10% to około 115 wyświetleń (zaokrąglając w górę).